In [1]:
import numpy as np
from scipy import signal, misc
import matplotlib.pyplot as plt
import mne
import scipy.io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix

import xgboost as xgb
import pickle
import os
import pandas as pd
import random
import shutil
from PIL import Image
import torch
torch.cuda.empty_cache()

data_path = r'../data/interim/dataset_1_cheb2/'
output_path = r'../data/processed/deep_learning_data/'

c:\Users\bcilab02\Documents\beka\thesis\thesis-venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
def divide_into_epochs(filename, s_data, epoch_length, output_path, data_ext='.npy'):
    start = 0
    end = epoch_length*500
    step = int((end-start)/2)
    count = 0
    while end < len(s_data):
        if 'depressed' in filename.lower():

            if data_ext == '.npy':
                file_path = output_path+'train/Depressed/'+filename[:-4]+str(count)+data_ext
                np.save(file_path, s_data[start:end].values.astype(np.double))
            if data_ext == '.jpg':
                file_path = output_path+'train_img/Depressed/'+filename[:-4]+str(count)+data_ext
                im = Image.fromarray(s_data[start:end].values, 'L')
                im.save(file_path)
        elif 'healthy' in filename.lower():
            
            if data_ext == '.npy':
                file_path = output_path+'train/Healthy/'+filename[:-4]+str(count)+data_ext
                np.save(file_path, s_data[start:end].values.astype(np.double))
            if data_ext == '.jpg':
                file_path = output_path+'train_img/Healthy/'+filename[:-4]+str(count)+data_ext
                im = Image.fromarray(s_data[start:end].values, 'L')
                im.save(file_path)
            
        count += 1
        start = start+step
        end = end+step

In [11]:
length = []
for filename in os.listdir(data_path):
    s_data = pd.read_csv(os.path.join(data_path, filename), index_col=False).iloc[:,2:-4]
    length.append(len(s_data))
    divide_into_epochs(filename, s_data, 6, output_path, '.npy')

In [12]:
# move 200 data from train folder ot test folder
def move_test_data(N, classname, output_path):
    data_list = os.listdir(output_path+'train/'+classname)
    test_data = random.sample(data_list, k=N)

    for file in test_data:
        shutil.move(output_path+'train/'+classname+'/'+file, output_path+'test/'+classname+'/'+file)

move_test_data(500, 'Depressed', output_path)
move_test_data(500, 'Healthy', output_path)

In [2]:
import torch
import torchvision
from torchvision import transforms
from torchvision.datasets import DatasetFolder
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torchvision.datasets import ImageFolder

def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

device = get_default_device()

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)


#train and test data directory
data_dir = output_path+"train_img/"
test_data_dir = output_path+"test_img/"

def npy_loader(path):
    sample = torch.from_numpy(np.load(path))
    return sample

class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

#load the train and test data
dataset = ImageFolder(
    root=data_dir,transform = transforms.Compose([
    transforms.Grayscale(1),
    transforms.ToTensor()
])
)
test_dataset = ImageFolder(
    root=test_data_dir,transform = transforms.Compose([
    transforms.Grayscale(1),
    transforms.ToTensor()
])
)

img, label = dataset[0]

print("Follwing classes are there : \n",dataset.classes)


def display_img(img,label):
    print(f"Label : {dataset.classes[label]}")
    plt.imshow(img.permute(0,2))

#display the first image in the dataset
#display_img(*dataset[0])

batch_size = 32
val_size = 2000
train_size = len(dataset) - val_size 

train_data,val_data = random_split(dataset,[train_size,val_size])
print(f"Length of Train Data : {len(train_data)}")
print(f"Length of Validation Data : {len(val_data)}")

#output
#Length of Train Data : 12034
#Length of Validation Data : 2000

#load the train and validation into batches.
train_loader = DataLoader(train_data, batch_size, shuffle = True, num_workers = 4)
val_loader = DataLoader(val_data, batch_size, num_workers = 4)

train_loader = DeviceDataLoader(train_loader, device)
val_loader = DeviceDataLoader(val_loader, device)

Follwing classes are there : 
 ['Depressed', 'Healthy']
Length of Train Data : 13905
Length of Validation Data : 2000


In [3]:
import torch.nn as nn
import torch.nn.functional as F

class ImageClassificationBase(nn.Module):
    
    def training_step(self, batch):
        images, labels = batch 
        images = torch.reshape(images, (-1, 1, 500, 31))
        out = self(images.double())                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        images = torch.reshape(images, (-1, 1, 500, 31))
        out = self(images.double())                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [9]:
class EEGClassification(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            
            nn.Conv2d(1, 16, kernel_size = (16,13), padding = 'same'),
            nn.ELU(),
            nn.BatchNorm2d(16),
            nn.AvgPool2d((6,6), padding=2),
            nn.Dropout(0.25),
            
            nn.Conv2d(16, 32, kernel_size = (12,5), padding = 'same'),
            nn.ELU(),
            nn.BatchNorm2d(32),
            nn.AvgPool2d((2,2)),
            nn.Dropout(0.25),

            nn.Conv2d(32, 64, kernel_size = (10,1), padding = 'same'),
            nn.ELU(),
            nn.BatchNorm2d(64),
            nn.AvgPool2d((2,2)),
            nn.Dropout(0.25),

            nn.Conv2d(64, 128, kernel_size = (8,1), padding = 'same'),
            nn.ELU(),
            nn.BatchNorm2d(128),
            nn.AvgPool2d((2,1)),
            nn.Dropout(0.25),

            nn.Conv2d(128, 256, kernel_size = (6,1), padding = 'same'),
            nn.ELU(),
            nn.BatchNorm2d(256),
            nn.Dropout(0.25),

            nn.Flatten(),
            nn.Linear(2560,1024),
            nn.Sigmoid(),
            nn.Linear(1024,2)
        )
    
    def forward(self, xb):
        return self.network(xb)

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

  

@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

  
def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD):
    
    history = []
    optimizer = opt_func(model.parameters(),lr)
    for epoch in range(epochs):
        model.double()
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    
    return history

num_epochs = 50
opt_func = torch.optim.Adam
lr = 0.001
model = EEGClassification()
model.to(device)
#fitting the model on training data and record the result after each epoch
history = fit(num_epochs, lr, model, train_loader, val_loader, opt_func)

Epoch [0], train_loss: 0.7109, val_loss: 0.9294, val_acc: 0.4816
Epoch [1], train_loss: 0.6735, val_loss: 0.6839, val_acc: 0.5565
Epoch [2], train_loss: 0.6787, val_loss: 0.6911, val_acc: 0.5446
Epoch [3], train_loss: 0.6770, val_loss: 2.6132, val_acc: 0.4633
Epoch [4], train_loss: 0.6753, val_loss: 0.6822, val_acc: 0.5565
Epoch [5], train_loss: 0.6739, val_loss: 0.6857, val_acc: 0.5491
Epoch [6], train_loss: 0.6738, val_loss: 1.9667, val_acc: 0.4836
Epoch [7], train_loss: 0.6761, val_loss: 1.6526, val_acc: 0.4742
Epoch [8], train_loss: 0.6739, val_loss: 0.6841, val_acc: 0.5129
Epoch [9], train_loss: 0.6746, val_loss: 0.6798, val_acc: 0.5565
Epoch [10], train_loss: 0.6739, val_loss: 0.6855, val_acc: 0.5565
Epoch [11], train_loss: 0.6744, val_loss: 0.6789, val_acc: 0.5565
Epoch [12], train_loss: 0.6760, val_loss: 0.6807, val_acc: 0.5541
Epoch [13], train_loss: 0.6744, val_loss: 0.7218, val_acc: 0.5446
Epoch [14], train_loss: 0.6742, val_loss: 0.6745, val_acc: 0.5689
Epoch [15], train_lo

KeyboardInterrupt: 

# Tensorflow model

In [1]:
import os
import re
# import warnings
# from sys import platform
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import CosineDecay
from tensorflow import keras
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, optimizers
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import math
import random

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7522478740914477701
xla_global_id: -1
]


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14582884875788460239
xla_global_id: -1
]


In [4]:
path = r'C:\Users\bcilab02\Documents\beka\thesis\MSc-Thesis\data\processed\deep_learning_data\train\Depressed/'
X = []
labels = []
for i in os.listdir(path):
    data = np.load(path+i)
    X.append(data)
    labels.append(0)

path = r'C:\Users\bcilab02\Documents\beka\thesis\MSc-Thesis\data\processed\deep_learning_data\train\Healthy/'
for i in os.listdir(path):
    data = np.load(path+i)
    X.append(data)
    labels.append(1)

kernels, chans, samples = 1, 31, 500
y = labels
temp = list(zip(X, y))
random.shuffle(temp)
X, y = zip(*temp)
# res1 and res2 come out as tuples, and so must be converted to lists.
X, y = list(X), list(y)
X = np.array(X)
y = np.array(y)
X = X[..., np.newaxis]
# take 50/25/25 percent of the data to train/validate/test
X_train      = X[0:1500,]
Y_train      = y[0:1500]
X_validate   = X[1500:,]
Y_validate   = y[1500:]

In [5]:
model=models.Sequential()
model.add(layers.Conv2D(16,(16,13),activation='elu',input_shape=(3000,31,1), padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.AveragePooling2D((6,6)))
model.add(layers.Dropout(rate=0.25))

model.add(layers.Conv2D(32,(12,5), activation='elu', padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.AveragePooling2D((4,4)))
model.add(layers.Dropout(rate=0.25))

model.add(layers.Conv2D(64,(10,1), activation='elu', padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.AveragePooling2D((3,1)))
model.add(layers.Dropout(rate=0.25))

model.add(layers.Conv2D(128,(8,1), activation='elu', padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.AveragePooling2D((2 ,1)))
model.add(layers.Dropout(rate=0.25))

model.add(layers.Conv2D(256,(6,1), activation='elu', padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.AveragePooling2D((2,1)))
model.add(layers.Dropout(rate=0.25))

model.add(layers.Flatten())
#model.add(layers.Dense(50, activation='elu'))
model.add(layers.Dense(2))
model.summary()


EPOCHS = 200
BATCH_SIZE = 20
step_per_epoch = math.ceil(X_train.shape[0]/BATCH_SIZE)
STEP_TOTAL = step_per_epoch * EPOCHS
INIT_LEARNING_RATE =  0.001

lr_schedule=CosineDecay(INIT_LEARNING_RATE,STEP_TOTAL,
                                                      alpha=0.0,
                                                      name=None)
optimizer =Adam(learning_rate=lr_schedule)

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy']
              )


history = model.fit(X_train,Y_train,
                    epochs=EPOCHS,
                    validation_data=(X_validate,Y_validate),
                    batch_size=BATCH_SIZE,
                    shuffle=True)


train_predictions = model.predict(x = X_train)
print(train_predictions)


test_predictions = model.predict(x = X_validate)
print(test_predictions)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 3000, 31, 16)      3344      
                                                                 
 batch_normalization (BatchN  (None, 3000, 31, 16)     64        
 ormalization)                                                   
                                                                 
 average_pooling2d (AverageP  (None, 500, 5, 16)       0         
 ooling2D)                                                       
                                                                 
 dropout (Dropout)           (None, 500, 5, 16)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 500, 5, 32)        30752     
                                                                 
 batch_normalization_1 (Batc  (None, 500, 5, 32)       1

KeyboardInterrupt: 